## Setup Reddit API and ElasticSearch Connection

In [2]:
reddit = praw.Reddit(client_id='Cz8OU1vxajnWDw',
                     client_secret='5qax29ZPI2_Rdjc1TsXXEypFduk',
                     redirect_uri='http://localhost:8080',
                     user_agent='my user agent')

print(reddit.auth.url(['identity'], '...', 'permanent'))

es = Elasticsearch([{'host':'localhost','port':9200}])
es

https://www.reddit.com/api/v1/authorize?client_id=Cz8OU1vxajnWDw&duration=permanent&redirect_uri=http%3A%2F%2Flocalhost%3A8080&response_type=code&scope=identity&state=...


<Elasticsearch([{'host': 'localhost', 'port': 9200}])>

## Grab popular subreddits from API

In [6]:
count = 0
for subreddit in reddit.subreddits.popular():
    count += 1
    doc = {
        "id": subreddit.display_name,
        "title": subreddit.title,
        "description": subreddit.public_description,
        "subscribers": subreddit.subscribers,
        "created": datetime.fromtimestamp(subreddit.created_utc).isoformat(),
        "language": subreddit.lang,
        "category": subreddit.advertiser_category,
        "redditors_indexed": False
    }
    
    # Ingest into Elastic
    es.index(index = 'subreddits.popular', id = subreddit.display_name, body = doc)
print(count)

100


## Grab up-to 1000 comments for each subreddit

In [7]:
redditors = 'redditors'

try:
    es.indices.create(index = redditors)
except:
    pass

results = scan(es,         
    query = {
        "query": {
            "term": {
                "redditors_indexed": {
                    "value": False
                }
            }
        }
    },
    index = 'subreddits.popular')

for subreddit_doc in results:    
    subreddit = reddit.subreddit(subreddit_doc['_id'])
    for comment in subreddit.comments(limit = 1000):
        doc = {}
        
        author = comment.author
        
        try:
            if not author or not author.name or not author.fullname:
                continue
        except:
            continue
        
        if (es.exists(index = redditors, id = author.name)):
            doc = es.get(index = redditors, id = author.name)['_source']
        
        if 'name' not in doc.keys():
            doc['name'] = author.name
            
        if 'object_id' not in doc.keys():
            doc['object_id'] = author.fullname
            
        if 'categories' not in doc.keys(): 
            doc['categories'] = []
        
        if subreddit.advertiser_category not in doc['categories']:
            doc['categories'].append(subreddit.advertiser_category)
        
        if 'subreddits' not in doc.keys(): 
            doc['subreddits'] = []
        
        if subreddit.display_name not in doc['subreddits']:
            doc['subreddits'].append(subreddit.display_name)
        
        if 'trophies' not in doc.keys():
            doc['trophies'] = []
        
        for trophy in author.trophies():
            if trophy.name not in doc['trophies']:
                doc['trophies'].append(trophy.name)
        
        es.index(index = redditors, id = author.name, body = doc)
    
    update = {
        "doc": {
              "redditors_indexed": True
        }
    }
    es.update(index = 'subreddits.popular', id = subreddit_doc['_id'], body = update)

NotFoundError: NotFoundError(404, 'search_phase_execution_exception', 'No search context found for id [861794]')